In [1]:
%%bash
cat /final/config_sample.json

{
	"Database User": "slucor",
	"Database Password": "S1UC0R",
	"Database Host": "localhost",
	"Database Name": "hds5210",

	"Start Date": "2016-11-01",
	"End Date": "2016-11-30",

	"Cost Field Mapping": {
		"MG001": ["ingredient_cost", "admin_fee", "rebate_amount"],
		"MG002": ["ingredient_cost"],
		"AZ001": ["incredient_cost", "admin_fee"]
	}
}


In [2]:
import json
def setup(filename):
    """(str) -> dict
    The function reads from the specified configuration file, in JSON format, and stores the contents
    in a Python dictionary.  That dictionary is returned to the caller, as shown in #4.
    
    >>> setup('/final/config_sample.json').get("Database User")
    'slucor'
    
    >>> setup('/final/config_sample.json').get("Cost Field Mapping")["MG001"][0]
    'ingredient_cost'
    
    >>> setup('/final/config_sample.json').get("Cost Field Mapping").get("AZ001")[1]
    'admin_fee'
    """
    #import json file so data is avaliable for use
    with open(filename) as file:
        jdata = json.load(file)
        return jdata
   
    result = setup('/final/config_sample.json')

In [3]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=3)

In [4]:
%%bash
cat /final/fees_sample.xml

<?xml version="1.0" encoding="UTF-8"?>

<fee_config>
  <fees member_group="MG001" mail_retail="MAIL">
    <admin_fee>0.50</admin_fee>
    <processing_fee>0.05</processing_fee>
  </fees>
  <fees member_group="MG001" mail_retail="RETAIL">
    <admin_fee>0.50</admin_fee>
    <processing_fee>0.10</processing_fee>
  </fees>
</fee_config>


In [5]:
import xml.etree.ElementTree as xml
def read_fees(filename):
    
    tree = xml.parse(filename)
    root = tree.getroot()
   
   # total data
    xmldict = {}
    for child in root.findall('fees'):
        attr = child.attrib
        memberGroupVal = attr.get('member_group')
       
    if memberGroupVal not in xmldict:   
        # judge MGOO1
        xmldict[memberGroupVal]={}
       
    #get data：{'admin_fee': '0.50', 'processing_fee': '0.10'}
    ADMIN_PROCESS = {}
    adminFeeVal = float(child.find('admin_fee').text)
    ADMIN_PROCESS["admin_fee"]=adminFeeVal
    processingFeeVal = float(child.find('processing_fee').text)
    ADMIN_PROCESS["processing_fee"]=processingFeeVal
       
    #get data：{'RETAIL': {'admin_fee': '0.50', 'processing_fee': '0.10'}}
    MAIL_RETAIL={}
    mailRetailVal = attr.get('mail_retail')
    MAIL_RETAIL[mailRetailVal] = ADMIN_PROCESS
       
    #get data：{'MG001': {'RETAIL': {'admin_fee': '0.50', 'processing_fee': '0.10'}}}
    xmldict[memberGroupVal].update(MAIL_RETAIL) 
    return xmldict  
    
#print(read_fees('fees_sample.xml'))

In [6]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=3)

In [7]:
def calc_total_cost(config, fees, member_group, mail_retail, ingredient_cost):
    #import files
    c = setup('/final/config.json')
    f = read_fees('/final/fees.xml')
    #pull data from files    
    equation = c.get("Cost Field Mapping").get(member_group)
    costs = f.get(member_group).get(mail_retail)
    if len(equation) == 1:
        print (ingredient_cost)
    elif len(equation) == 3:
        print (round (ingredient_cost + cost['admin fee'] + costs['processing fee']))
    else:
        if 'admin fee' in costs:
            print (round (ingredient_cost + costs['admin fee']), 5)
        elif 'processing fee' in costs:
            print (round(ingredient_cost + costs['processing fee']), 5)
    

In [8]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=3)

In [9]:
import pymysql.cursors

In [10]:
c = setup('/final/config.json')
f = read_fees('/final/fees.xml')

connection = pymysql.connect(
    host=c.get("Database Host"),
    user=c.get("Database User"),
    password=c.get("Database Password"),
    db=c.get("Database Name"),
    cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()
sql = 'SELECT * from claims'
cursor.execute(sql)
result = cursor.fetchall()
connection.close



<bound method Connection.close of <pymysql.connections.Connection object at 0x7f663854b3c8>>

In [11]:
result

[{'benefit_plan_code': 'PZU',
  'brand_generic_ind': 'GENERIC',
  'business_unit_id': 'HNCA',
  'copay_amount': Decimal('5.00000'),
  'drug_ndc': '60505036302',
  'ingredient_cost': Decimal('88.98000'),
  'lob_code': '',
  'lob_desc': '',
  'mail_order_ind': 'RETAIL',
  'member_group': '00400F',
  'new_refill_indicator': 'NEW',
  'pa_indicator': 'N',
  'paid_amount': Decimal('88.23000'),
  'process_date': '2015-01-05',
  'rx_count': '',
  'rx_number': '1460346',
  'sales_tax': Decimal('0.00000')},
 {'benefit_plan_code': 'PD3',
  'brand_generic_ind': 'GENERIC',
  'business_unit_id': 'HNCA',
  'copay_amount': Decimal('10.00000'),
  'drug_ndc': '58177000104',
  'ingredient_cost': Decimal('20.43000'),
  'lob_code': '',
  'lob_desc': '',
  'mail_order_ind': 'MAIL',
  'member_group': '00400F',
  'new_refill_indicator': 'NEW',
  'pa_indicator': 'N',
  'paid_amount': Decimal('18.18000'),
  'process_date': '2015-01-05',
  'rx_count': '',
  'rx_number': '30567',
  'sales_tax': Decimal('0.00000')